In [28]:
# Pandas is used for data manipulation
import pandas as pd
import numpy as np

# Read in data and display first 5 rows
z_data = pd.read_csv('esem_act_deact_zscore_groups.csv',sep=';',decimal='.')
z_data.drop(list(z_data.filter(regex = '_deact')),axis = 1, inplace = True)
z_data.drop(list(z_data.filter(regex = 'aggr')),axis = 1, inplace = True)
data = z_data

#we need the list with the labels to test the predictions; everytime the label or proband changes, we add it to the list
# grouped = z_data.groupby(['proband', 'trial','task'])
# groupedAgg = grouped.aggregate(np.mean)
labelsComputingAccuracy = z_data.groupby(['proband', 'trial','task']).aggregate(np.mean).index.get_level_values(level='task')
labels = z_data.task

#=== remove everything that is not a feature ===================================================
z_data = z_data.drop('proband', axis = 1)
z_data = z_data.drop('scan', axis = 1)
z_data = z_data.drop('trial', axis = 1)
z_data = z_data.drop('task', axis = 1)
z_data = z_data.drop('snippet', axis = 1)
z_data = z_data.drop('response', axis = 1)

#=== creating training and validation set ======================================================
# Saving feature names for later use
feature_list = z_data.columns

# Convert to numpy array
features = np.array(z_data)
labels = np.array(labels)

numLabels = 3 # num of different labels (comprehension and rest)
sizeTrainSet = 13 # num of participants used for training
sizeTestSet = 4 # num participants used for testing
 
# get the number of trials per participant
numScans = len(data.scan.unique())
numParticipants = sizeTrainSet + sizeTestSet


#get the rows excluding the last indexed (i.e., row with the index 587 is the last to include)
training_features = features[0:numScans*sizeTrainSet]
training_features = np.array(training_features)
testing_features = features[numScans*sizeTrainSet:numScans*numParticipants]
testing_features = np.array(testing_features)

training_target = labels[0:numScans*sizeTrainSet]
training_target = np.array(training_target)
testing_target = labels[numScans*sizeTrainSet:numScans*numParticipants]
testing_target = np.array(testing_target)



C:\Users\author\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1155,1156,1157,1158,1159,1160,1161,1162,1163,1164) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(training_features.shape)
print(testing_features.shape)
print(training_target.shape)
print(testing_target.shape)
print(len(labelsComputingAccuracy))
type(training_features)
z_data.head(5)

(8710, 888)
(2680, 888)
(8710,)
(2680,)
1020


,140-99-73_act,140-102-73_act,140-105-73_act,140-108-73_act,140-99-76_act,140-102-76_act,140-105-76_act,140-108-76_act,140-99-79_act,140-102-79_act,...,194-162-127_act,194-165-127_act,194-168-127_act,194-171-127_act,194-174-127_act,194-165-130_act,194-168-130_act,194-171-130_act,194-174-130_act,194-171-133_act
0,-1.161421,-0.923320,-0.320038,0.629510,-0.136895,-0.536765,-0.514446,-0.463186,-0.502790,-0.892422,...,-0.203277,-1.459860,-2.242707,-2.676553,-1.924082,-2.018515,-1.829216,-2.202872,-2.132474,0.467849
1,-1.257476,-0.909639,-0.472218,0.218840,-0.140112,-0.213969,-0.352091,-0.384465,-0.116147,-0.223085,...,0.352071,-0.933596,-1.284921,-1.191942,-1.035631,-1.289443,-0.941265,-1.018077,-1.417968,1.748105
2,-0.537064,-0.300862,-0.773474,-1.240208,0.297384,0.712767,0.028945,-0.170796,0.775871,1.227146,...,0.018043,-1.058567,-1.090775,-0.893717,-0.827641,-0.981069,-0.588830,-0.795635,-1.381101,2.291829
3,-0.387332,-0.971201,-1.658605,-1.580764,-0.272004,-1.237891,-1.372606,-0.676855,0.431850,0.328499,...,0.850040,-0.110181,0.046920,0.412480,0.480642,-0.282631,-0.268435,-0.420997,-0.522641,1.154251
4,-0.000287,-0.454766,-1.282812,-1.347049,0.593337,-0.040423,-0.517759,-0.455688,1.549156,1.456456,...,-0.010647,-0.660050,0.000325,0.770610,0.682213,-0.429670,-0.060178,-0.060407,-0.294420,1.755818


In [ ]:
from tpot import TPOTClassifier
fileNameTPot = 'Act/tpot_mnist_pipeline_NotAggregatedParticipantSplit_Act'

tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2,n_jobs = 20)
tpot.fit(training_features, training_target)
print(tpot.score(testing_features, testing_target))
tpot.export(fileNameTPot + '.py')

Generation 1 - Current best internal CV score: 0.5398392652123996



Best pipeline: KNeighborsClassifier(input_matrix, n_neighbors=51, p=1, weights=uniform)


In [35]:
with open(fileNameTPot + '.py') as f:
    content = f.readlines()

cleanedContent = []
for line in content:
    if 'tpot_data' not in line and 'training_target, testing_target' not in line:
        cleanedContent.append(line)

fileForLearning = fileNameTPot + '_cleaned.py'
with open(fileForLearning, 'w') as filehandle:  
    for line in cleanedContent:
        filehandle.write('%s\n' % line)

In [36]:
%run -i $fileForLearning

In [37]:
z_data = data
condition = z_data.at[sizeTrainSet*numScans, 'task']
participant = z_data.at[sizeTrainSet*numScans, 'proband']
majorityVote = []
countComprehension = 0
countRest = 0
countD2 = 0
    
for i,j in zip(range(sizeTrainSet*numScans,len(z_data)),range(len(results))):
    if condition != z_data.at[i, 'task'] or participant != z_data.at[i, 'proband']:
        condition = z_data.at[i, 'task']
        participant = z_data.at[i, 'proband']
        if (countComprehension > countRest) and (countComprehension > countD2):
            majorityVote.append('C')
        elif countD2 > countRest and countD2 > countComprehension:
                majorityVote.append('D2')
        elif countRest > countComprehension and countRest > countD2:
            majorityVote.append('R')
        elif countComprehension == countRest:
            majorityVote.append('CR')
        elif countComprehension == countD2:
            majorityVote.append('CD2')
        else:
            majorityVote.append('D2R')
        countComprehension = 0
        countRest = 0
        countD2 = 0

    if (results[j] == 0):
        countComprehension = countComprehension + 1
    if (results[j] == 1):
        countRest = countRest + 1
    if (results[j] == 2):
        countD2 = countD2 + 1

if (results[j] == 0):
    countComprehension = countComprehension + 1
if (results[j] == 1):
    countRest = countRest + 1
if (results[j] == 2):
    countD2 = countD2 + 1

if (countComprehension > countRest) and (countComprehension > countD2):
    majorityVote.append('C')
elif countD2 > countRest and countD2 > countComprehension:
        majorityVote.append('D2')
elif countRest > countComprehension and countRest > countD2:
    majorityVote.append('R')
elif countComprehension == countRest:
    majorityVote.append('CR')
elif countComprehension == countD2:
    majorityVote.append('CD2')
else:
    majorityVote.append('D2R')


In [44]:
labelsComputingAccuracy = pd.Series(labelsComputingAccuracy)
labelsComputingAccuracy = labelsComputingAccuracy.replace({0: 'C', 1: 'R', 2: 'D2'})

num_matches = 0;
for a, b in zip(majorityVote, labelsComputingAccuracy):
    if b in a:
        num_matches = num_matches + 1
print('Number of matches:',num_matches,'(of',len(labelsComputingAccuracy),')')

accuracy = num_matches/len(labelsComputingAccuracy)*100
print('Accuary: ',accuracy)


Number of matches: 99 (of 1020 )
Accuary:  9.705882352941178


In [50]:
with open('listOfAccuracies.txt','a+') as f:
    f.write('%s\n' % 'NoAgg_Act' + str(accuracy) + '\n')
